In [1]:
## This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mltiverse-join-comp/sample_submission.csv
/kaggle/input/mltiverse-join-comp/train.csv
/kaggle/input/mltiverse-join-comp/test.csv


importing data

In [2]:
train_data=pd.read_csv('/kaggle/input/mltiverse-join-comp/train.csv')
test_data=pd.read_csv('/kaggle/input/mltiverse-join-comp/test.csv')
sample=pd.read_csv('/kaggle/input/mltiverse-join-comp/sample_submission.csv')

making some comparisions

In [3]:
sample.shape

(16710, 2)

In [4]:
test_data.shape

(16710, 65)

checking if photo id is the same for both

In [5]:
(test_data['Photo'] == sample['Photo']).all()

True

therefore it is the exact same

checking the number of rows and analysing test data

In [6]:
train_data.shape

(87642, 67)

In [7]:
train_data['Photo'].nunique()

74502

In [8]:
 train_data[['Photo', 'year']].drop_duplicates().shape[0]

87642

keeping only one row per id(the latest year in this case)

first and foremost let me check for test data. the shape, unique ids, photo etc etc


In [9]:
test_data.shape[0]==test_data['ID'].nunique()

True

therefore the ids and photos are linearly mapped, so should only keep one id for training data

In [10]:
# Sort the train_data DataFrame by 'ID' and 'year' in descending order
train_data_sorted = train_data.sort_values(by=['ID', 'year'], ascending=[True, False])

# Drop duplicate 'ID' values, keeping only the first occurrence (which corresponds to the latest year)
train_data_1 = train_data_sorted.drop_duplicates(subset='ID', keep='first')

# Reset the index of the new dataset
train_data_1 = train_data_1.reset_index(drop=True)

In [11]:
train_data_1.shape

(33699, 67)

repeating the same process as first submission now

In [12]:
train_1=train_data_1[['Potential','Overall','Wage','International Reputation','Age']]
test_1=test_data[['Potential','Overall','Wage','International Reputation','Age']]

train feature

In [13]:
y_train=train_data_1['Value']

converting Value and Wage as per

In [14]:
import re
# Define a function to preprocess the 'Wage' column
def preprocess_wage(value):
    value = re.sub(r'€', '', value)  # Remove euro symbol
    if 'K' in value:
        value = float(re.sub(r'[^0-9.]', '', value)) * 1000  # Remove 'K' and multiply by 1000
    elif 'M' in value:
        value = float(re.sub(r'[^0-9.]', '', value)) * 1000000  # Remove 'M' and multiply by 1000000
    else:
        value = float(value)
    return value

# Apply the preprocessing function to 'Wage' column in train_1 dataset
train_1['Wage'] = train_1['Wage'].apply(preprocess_wage)

# Convert the column to float data type
train_1['Wage'] = train_1['Wage'].astype(float)

# Apply the preprocessing function to 'Wage' column in train_1 dataset
test_1['Wage'] = test_1['Wage'].apply(preprocess_wage)

# Convert the column to float data type
test_1['Wage'] = test_1['Wage'].astype(float)

#Applying it to y_train too
y_train=y_train.apply(preprocess_wage)

#Convert the column to float data type
y_train=y_train.astype(float)

/tmp/ipykernel_20/3838849759.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_1['Wage'] = train_1['Wage'].apply(preprocess_wage)
/tmp/ipykernel_20/3838849759.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_1['Wage'] = train_1['Wage'].astype(float)
/tmp/ipykernel_20/3838849759.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

fitting model with  best parameters

In [15]:
from xgboost import XGBRegressor
model = XGBRegressor(learning_rate=0.05, n_estimators=500)
model.fit(train_1, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=500, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [16]:
pred=model.predict(test_1)

In [17]:
output = pd.DataFrame({'Photo':test_data['Photo'],'Value': pred})  # Create the DataFrame with the 'Value' column
output.head()

,Photo,Value
0,https://cdn.sofifa.com/players/212/198/22_60.png,68386456.0
1,https://cdn.sofifa.com/players/209/658/22_60.png,52658680.0
2,https://cdn.sofifa.com/players/176/580/22_60.png,26261282.0
3,https://cdn.sofifa.com/players/192/985/22_60.png,84744840.0
4,https://cdn.sofifa.com/players/224/334/22_60.png,28591482.0


hence output submission has been created

converting to a csv as needed

In [18]:
output.to_csv('submissions.csv',index=False)
print("Submission saved successfully")

Submission saved successfully
